<a href="https://colab.research.google.com/github/prj-07/Bagofwordscode/blob/main/BagOfWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import spacy

In [ ]:
df = pd.read_csv("spam_ham_dataset.csv")

In [ ]:
df.head(5)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [ ]:
spam =0
for i in df['label_num']:
  if(i==1):
    spam +=1

spam    

1499

In [ ]:
df.shape

(5171, 4)

In [ ]:
length = len(df)

In [ ]:
unspam = length - spam

In [ ]:
unspam

3672

In [ ]:
y = df['label_num']
x = df['text']

##data imbalance handling

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
min_samples = 1499 ##since 250 non spam data present
df_spam = df[df.label_num==0].sample(min_samples, random_state=2022)
df_unspam = df[df.label_num==1].sample(min_samples, random_state=2022)

In [ ]:
df_balanced = pd.concat([df_spam,df_unspam],axis=0)

In [ ]:
df_balanced.label.value_counts()

ham     1499
spam    1499
Name: label, dtype: int64

In [ ]:
##huraay ! we just balanced the data

##lets train

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(df_balanced.text,df_balanced.label_num,test_size = 0.2)

In [ ]:
xtrain.values

array(['Subject: ( no subject )\r\nhey daren its me .\r\nhave you had any sleep lately ? ? ? ?\r\nsee ya at the beach .',
       'Subject: no more injections\r\n',
       'Subject: out of office\r\nplease note that i will be out of the office on friday , 10 / 6 / 00 . in my\r\nabsence tom acton will be handling the oasis & lonestar interconnects at\r\nkaty , and mary poorman will handle the nomination to oasis and cornhusker if\r\nnecessary , but it is already nominated for the weekend .\r\nthanks ,\r\nmark\r\nx 33396',
       ...,
       'Subject: best software prices . incorrectness chivalrousness\r\ncatch make see turn . list be still lake voice tire . large did\r\nthan , can . smile , energy field well . before tree cover three\r\nthought , may . chord never thing . hair box multiply they set .\r\nfarm hear way group us say . climb sudden mother , take stead ,\r\nsubject .\r\n',
       'Subject: buy xana : x inexpen : sive pills now yjppppdz\r\nhi again ,\r\nwe now have many meds a

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(xtrain.values)
X_train_cv

<2398x38946 sparse matrix of type '<class 'numpy.int64'>'
	with 224664 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
X_train_cv.shape

(2398, 38946)

In [ ]:
v.get_feature_names_out()[1771]

'568'

In [ ]:
v.get_feature_names_out()[177]

'10126'

In [ ]:
v.vocabulary_ ##wo sara vocab jo input email me hoga uska list ban 

In [ ]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
import numpy as np

In [ ]:
np.where(X_train_np[0]!=0) ##ye saare index par value non zeroo hai ie 1 hai ie wo spam hai

(array([ 4556,  5193,  6048, 11507, 17875, 18127, 18448, 20560, 21950,
        23671, 25447, 31674, 32546, 33694, 34697, 38379, 38565]),)

##using  naive bayes for training

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, ytrain)

MultinomialNB()

## doubt - why not xtest.values? why not v.fit_transfrom like before? in model why x_train_cv ? why not x_train_cv.value?


In [ ]:
X_test_cv = v.transform(xtest)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       286
           1       0.98      0.96      0.97       314

    accuracy                           0.97       600
   macro avg       0.97      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600



In [ ]:
emails = [
    'Hey mohan, lets eat smek?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([1, 1])